# Importação das Bibliotecas

In [1]:
import cv2
import json
import numpy as np
import os

# Seleção das Vagas

In [35]:
# leitura da imagem
while True:
    entrada = input('Nome do arquivo (com extensão, ex: imagem.png): ')
    
    if entrada == 'x':
        print("Encerrado")
        break

    if os.path.isfile(entrada):
        img = cv2.imread(entrada)
        if img is not None:
            break
        else:
            print("Erro ao abrir a imagem. Tente outro arquivo.")
    else:
        print("Arquivo não encontrado. Tenta de novo.")

vaga_atual = []
todas_vagas = []

def clique(event, x, y, flags, param):
    global vaga_atual, todas_vagas

    if event == cv2.EVENT_LBUTTONDOWN: # se o botão esquerdo for pressionado
        vaga_atual.append((x, y)) # adiciona o ponto à lista da vaga atual
        # desenhar o ponto onde o usuário clicou (círculo azul)
        cv2.circle(img_clone, (x, y), 5, (255, 0, 0), -1)  # desenha um círculo azul onde o usuário clicou (extremidade da vaga)
        print(f"Ponto clicado: {(x, y)}") # printa o ponto clicado

    if len(vaga_atual) == 2: # se o usuário já fez 2 cliques (preencheu uma vaga):
        todas_vagas.append(vaga_atual.copy()) # salva a vaga na lista de vagas, que contém as coordenadas de cada vaga
        cv2.rectangle(img_clone, vaga_atual[0], vaga_atual[1], (255, 0, 0), 2) # desenha um retângulo azul, mostrando onde a vaga foi selecionada
        print(f"Vaga salva: {vaga_atual}") # printa os pontos da vaga, mostrando que ela foi lida
        vaga_atual = []


img = cv2.imread(entrada)
img_clone = img.copy()
cv2.namedWindow("Imagem")
cv2.setMouseCallback("Imagem", clique)

while True:
    cv2.imshow("Imagem", img_clone)
    key = cv2.waitKey(1)
    if key == 27:
        break # sai do loop se ESC for pressionado

cv2.destroyAllWindows()

with open("coordenadas_vagas.json", "w") as f:
    json.dump(todas_vagas, f)

Ponto clicado: (11, 18)
Ponto clicado: (34, 80)
Vaga salva: [(11, 18), (34, 80)]
Ponto clicado: (49, 18)
Ponto clicado: (71, 81)
Vaga salva: [(49, 18), (71, 81)]
Ponto clicado: (80, 18)
Ponto clicado: (103, 83)
Vaga salva: [(80, 18), (103, 83)]
Ponto clicado: (111, 21)
Ponto clicado: (134, 87)
Vaga salva: [(111, 21), (134, 87)]
Ponto clicado: (141, 21)
Ponto clicado: (166, 92)
Vaga salva: [(141, 21), (166, 92)]
Ponto clicado: (171, 21)
Ponto clicado: (200, 93)
Vaga salva: [(171, 21), (200, 93)]
Ponto clicado: (203, 22)
Ponto clicado: (233, 90)
Vaga salva: [(203, 22), (233, 90)]
Ponto clicado: (232, 23)
Ponto clicado: (265, 89)
Vaga salva: [(232, 23), (265, 89)]
Ponto clicado: (266, 22)
Ponto clicado: (294, 91)
Vaga salva: [(266, 22), (294, 91)]
Ponto clicado: (295, 22)
Ponto clicado: (327, 97)
Vaga salva: [(295, 22), (327, 97)]
Ponto clicado: (326, 25)
Ponto clicado: (360, 103)
Vaga salva: [(326, 25), (360, 103)]
Ponto clicado: (355, 25)
Ponto clicado: (389, 85)
Vaga salva: [(355, 25),

# Análise das Vagas

In [36]:
img = cv2.imread(entrada)
with open("coordenadas_vagas.json", "r") as f:
    vagas = json.load(f)

livres = 0
ocupadas = 0

def calcular_entropia(imagem):
    histograma = cv2.calcHist([imagem], [0], None, [256], [0, 256])
    # 0 -> primeiro canal da imagem; None -> não há aplicação de máscara; 256 -> nbins; 0,256 -> intervalo de intensidade
    histograma = histograma / histograma.sum()  # normaliza o histograma dividindo pelo somatório total para garantir que a soma dos valores seja 1
    entropia = -np.sum(histograma * np.log2(histograma + 0.0000001)) # (evitando log(0))
    return entropia

for i, vaga in enumerate(vagas):
    (x1, y1), (x2, y2) = vaga
    vaga_crop = img[y1:y2, x1:x2]

    cinza = cv2.cvtColor(vaga_crop, cv2.COLOR_BGR2GRAY) # converte a imagem da vaga para a grayscale

    entropia = calcular_entropia(cinza) # calcula a entropia da imagem da vaga em grayscale usando a função criada anteriormente
    print(f"Vaga {i+1}: entropia = {entropia:.2f}") # mostra a entropia da vaga analisada

# definimos o valor 6.25 como sendo adequado para definir a situação das vagas analisadas (valor pode variar conforme imagem analisada)
    if entropia < 6.25:
        cor = (0, 255, 0) # cor verde -> vaga livre
        livres +=1
    else:
        cor = (0, 0, 255) # cor vermelha -> vaga ocupada
        ocupadas += 1

    cv2.rectangle(img, (x1, y1), (x2, y2), cor, 2) # desenha um retângulo nas coordenadas definidas inicialmente, com a coloração correspondente

cv2.imshow("Resultado", img)
cv2.imwrite("resultado.png", img)

while True:
    cv2.imshow("Resultado", img)
    key = cv2.waitKey(1)
    if key == 27: 
        break # sai do loop se ESC for pressionado

cv2.destroyAllWindows()

Vaga 1: entropia = 6.37
Vaga 2: entropia = 5.57
Vaga 3: entropia = 5.18
Vaga 4: entropia = 5.40
Vaga 5: entropia = 6.95
Vaga 6: entropia = 6.87
Vaga 7: entropia = 6.95
Vaga 8: entropia = 5.34
Vaga 9: entropia = 5.02
Vaga 10: entropia = 5.87
Vaga 11: entropia = 6.91
Vaga 12: entropia = 6.65
Vaga 13: entropia = 6.01
Vaga 14: entropia = 6.69
Vaga 15: entropia = 6.36
Vaga 16: entropia = 6.78
Vaga 17: entropia = 6.71
Vaga 18: entropia = 7.12
Vaga 19: entropia = 5.65
Vaga 20: entropia = 5.43
Vaga 21: entropia = 6.85
Vaga 22: entropia = 6.90
Vaga 23: entropia = 5.85
Vaga 24: entropia = 5.25
Vaga 25: entropia = 7.21
Vaga 26: entropia = 6.00
Vaga 27: entropia = 6.93
Vaga 28: entropia = 6.15
Vaga 29: entropia = 5.93
Vaga 30: entropia = 6.06
Vaga 31: entropia = 5.94
Vaga 32: entropia = 6.97
Vaga 33: entropia = 6.20
Vaga 34: entropia = 6.52
Vaga 35: entropia = 7.04
Vaga 36: entropia = 6.46
Vaga 37: entropia = 5.26
Vaga 38: entropia = 6.66
Vaga 39: entropia = 5.35
Vaga 40: entropia = 5.09
Vaga 41: 

# Relatório

In [37]:
relatorio = []

# escreve no relatório
relatorio.append(f"Total de vagas: {livres + ocupadas}")
relatorio.append(f"Vagas livres: {livres}")
relatorio.append(f"Vagas ocupadas: {ocupadas}")

# salva o relatório
with open("relatorio_vagas.txt", "w") as f:
    f.write("\n".join(relatorio))


# Foto com Legenda

In [28]:
img_legenda = cv2.imread("resultado.png")

total = livres + ocupadas
texto = [
    f"Total: {total}",
    f"Livres: {livres}",
    f"Ocupadas: {ocupadas}"
]

# fundo branco
overlay = img_legenda.copy()
cv2.rectangle(overlay, (10, img_legenda.shape[0]-50), (220, img_legenda.shape[0]-10), (255, 255, 255), -1)
img_legenda = cv2.addWeighted(overlay, 0.6, img_legenda, 0.4, 0)

# escrever texto com fonte menor
for i, linha in enumerate(texto):
    y = img_legenda.shape[0] - 35 + i*15
    cv2.putText(img_legenda, linha, (20, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)

cv2.imwrite("legenda.png", img_legenda)

True

# Método para Média da Entropia

In [38]:
img = cv2.imread(entrada)  # lê a imagem
with open("coordenadas_vagas.json", "r") as f:
    vagas = json.load(f)  # lê o json com as coordenadas das vagas definidas

livres = 0
ocupadas = 0
entropias = []

def calcular_entropia(imagem):
    histograma = cv2.calcHist([imagem], [0], None, [256], [0, 256])
    histograma = histograma / histograma.sum()
    entropia = -np.sum(histograma * np.log2(histograma + 1e-7))
    return entropia

# Primeiro loop: coleta as entropias
for vaga in vagas:
    (x1, y1), (x2, y2) = vaga
    vaga_crop = img[y1:y2, x1:x2]
    cinza = cv2.cvtColor(vaga_crop, cv2.COLOR_BGR2GRAY)
    entropias.append(calcular_entropia(cinza))

# Define o limiar com base na média + deslocamento
limiar = np.mean(entropias)
print(f"Limiar baseado na média das entropias: {limiar:.2f}")

# Segundo loop: classifica com base no limiar adaptativo
for i, vaga in enumerate(vagas):
    (x1, y1), (x2, y2) = vaga
    vaga_crop = img[y1:y2, x1:x2]
    cinza = cv2.cvtColor(vaga_crop, cv2.COLOR_BGR2GRAY)
    entropia = calcular_entropia(cinza)
    print(f"Vaga {i+1}: entropia = {entropia:.2f}")

    if entropia < limiar:
        cor = (0, 255, 0)
        livres += 1
    else:
        cor = (0, 0, 255)
        ocupadas += 1

    cv2.rectangle(img, (x1, y1), (x2, y2), cor, 2)

cv2.imshow("Resultado", img)
cv2.imwrite("resultado.png", img)

while True:
    cv2.imshow("Resultado", img) # mostra a imagem
    key = cv2.waitKey(1)
    if key == 27:
        break

cv2.destroyAllWindows()

Limiar baseado na média das entropias: 6.36
Vaga 1: entropia = 6.37
Vaga 2: entropia = 5.57
Vaga 3: entropia = 5.18
Vaga 4: entropia = 5.40
Vaga 5: entropia = 6.95
Vaga 6: entropia = 6.87
Vaga 7: entropia = 6.95
Vaga 8: entropia = 5.34
Vaga 9: entropia = 5.02
Vaga 10: entropia = 5.87
Vaga 11: entropia = 6.91
Vaga 12: entropia = 6.65
Vaga 13: entropia = 6.01
Vaga 14: entropia = 6.69
Vaga 15: entropia = 6.36
Vaga 16: entropia = 6.80
Vaga 17: entropia = 6.71
Vaga 18: entropia = 7.12
Vaga 19: entropia = 5.65
Vaga 20: entropia = 5.43
Vaga 21: entropia = 6.85
Vaga 22: entropia = 6.90
Vaga 23: entropia = 5.85
Vaga 24: entropia = 5.25
Vaga 25: entropia = 7.21
Vaga 26: entropia = 6.00
Vaga 27: entropia = 6.93
Vaga 28: entropia = 6.15
Vaga 29: entropia = 5.93
Vaga 30: entropia = 6.06
Vaga 31: entropia = 5.94
Vaga 32: entropia = 6.97
Vaga 33: entropia = 6.20
Vaga 34: entropia = 6.52
Vaga 35: entropia = 7.04
Vaga 36: entropia = 6.46
Vaga 37: entropia = 5.26
Vaga 38: entropia = 6.66
Vaga 39: entrop